In [39]:
import os
import time
import random

import timm
import torch
import augraphy as ag
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.optim import AdamW
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from PIL import ImageOps
import re 
from PIL import ImageEnhance, ImageFilter
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import pytesseract
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD


In [ ]:
#메모리 초기화
torch.cuda.empty_cache()

In [5]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [6]:
# 데이터셋 클래스를 정의합니다.
#oversampling -> 클래스 빈 130개 채워서 1700개 : max_oversample 로 배율 조정
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None, oversample=False, class_transforms=None, max_oversample=4, use_ocr=False):
        self.df = pd.read_csv(csv)
        self.path = path
        self.transform = transform
        self.class_transforms = class_transforms  # 추가된 부분
        self.oversample = oversample
        self.max_oversample = max_oversample
        self.use_ocr = use_ocr

        # OCR 대상 클래스 텍스트 추출 및 벡터화
        if self.use_ocr:
            ocr_texts = [
                extract_text_from_image(os.path.join(self.path, row['ID'])) 
                if row['target'] in [3, 4, 7, 14] else "" 
                for _, row in self.df.iterrows()
            ]
            self.text_vectors = text_to_vector(ocr_texts)
        else:
            self.text_vectors = [np.zeros(50) for _ in range(len(self.df))]  # OCR 비활성화 시 기본 0 벡터 사용
            
        if self.oversample:
            self.df, self.text_vectors = self.apply_oversampling(self.df, self.text_vectors)

    def apply_oversampling(self, df, text_vectors):
        class_counts = Counter(df['target'])
        max_count = int(max(class_counts.values()) * self.max_oversample)  # 설정한 배수만큼 샘플 수 제한
        oversampled_df = df.copy()
        oversampled_text_vectors = text_vectors[:]

        for cls, count in class_counts.items():
            if count < max_count:
                # 부족한 샘플 수만큼 추가 복제
                samples_to_add = df[df['target'] == cls]
                text_vectors_to_add = [text_vectors[i] for i in samples_to_add.index]
                
                for _ in range(max_count // count - 1):  # 배수만큼 추가
                    oversampled_df = pd.concat([oversampled_df, samples_to_add])
                    oversampled_text_vectors.extend(text_vectors_to_add)
                
                # 나머지 추가 복제
                remainder = max_count % count
                if remainder > 0:
                    oversampled_df = pd.concat([oversampled_df, samples_to_add.sample(remainder, replace=True)])
                    oversampled_text_vectors.extend(text_vectors_to_add[:remainder])

        return oversampled_df.sample(frac=1).reset_index(drop=True), oversampled_text_vectors

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        image_name = self.df.iloc[idx]['ID']
        target = self.df.iloc[idx]['target']
        img_path = os.path.join(self.path, image_name)
        img = np.array(Image.open(img_path).convert("RGB"))
        
        # 클래스별로 다른 transform 적용
        if self.class_transforms and target in self.class_transforms:
            img = self.class_transforms[target](image=img)['image']  # 변경된 부분
        elif self.transform:
            img = self.transform(image=img)['image']
        
        # OCR 텍스트 벡터 가져오기
        text_vector = self.text_vectors[idx] if self.use_ocr and target in [3, 4, 7, 14] else np.zeros(50)
        
        return img, torch.tensor(text_vector, dtype=torch.float32), target

In [53]:
import cv2

def preprocess_image_for_ocr(img):
    # 이미지 확대
    img = img.resize((img.width * 2, img.height * 2), Image.LANCZOS)
    
    # 밝기 및 대비 조절
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2)  # 대비 증가
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(1.5)  # 밝기 증가
    
    # 이미지 날카로움 증가
    img = img.filter(ImageFilter.SHARPEN)

    # 그레이스케일 및 이진화
    img = img.convert('L')  # 그레이스케일로 변환
    img = np.array(img)
    _, img = cv2.threshold(img, 140, 255, cv2.THRESH_BINARY)  # 이진화 처리

    return Image.fromarray(img)

def extract_text_from_image(image_path):
    # 이미지 열기 및 전처리
    img = Image.open(image_path).convert("RGB")
    img = preprocess_image_for_ocr(img)  
    # OCR로 텍스트 추출 (한국어+영어)
    text = pytesseract.image_to_string(img, lang='kor', config='--psm 6')
    # 특수 문자 제거 (필요에 따라 조정)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)  
    return text.strip()

def text_to_vector(texts):
    vectorizer = TfidfVectorizer(max_features=500)
    text_vectors = vectorizer.fit_transform(texts)
    
    # 특징 개수에 맞춰 `n_components` 동적으로 설정
    n_components = min(50, text_vectors.shape[1])  # 최대 50차원으로 축소
    svd = TruncatedSVD(n_components=n_components)
    
    text_vectors = svd.fit_transform(text_vectors)
    return text_vectors

In [55]:
# Tesseract 실행 파일 경로 설정 (예시)
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# 특정 이미지 경로 지정 (예: OCR 처리를 확인할 이미지 파일 이름)
image_path = "/root/data/train/0d6a14437ad1a20e.jpg"

# OCR 처리된 텍스트 추출 및 출력
extracted_text = extract_text_from_image(image_path)
print("Extracted OCR Text:")
print(extracted_text)

# 텍스트 벡터화
text_vector = text_to_vector([extracted_text])[0]  # 벡터화 후 첫 번째 결과 가져오기
print("\nText Vector (First 10 values):")
print(text_vector[:10])  # 첫 10개의 벡터 값만 확인


Extracted OCR Text:
진 료 확 인 서
차 트 번 호
연 번 호  르
며 늬 네     술
신웜뿌리뭄뭄솜 동 반 한 요 주 및 기타 주 간 판 장 애 65519 951
 변 훨 일 까 지 2 
2023 년 06 월 15 일 부 터
2023 년 06 뭘 15 일 까 지 1  일 간 
실 총 원 열 자  15 일 콜 
 홍7l외 같 이 진 료 받 았 음 을 확 연 힙 니 다  2
발 행 일 2023 년 06 월 27 일
의 사 성 영
면 허 번 호
   물 000000  7 으 
전 화 변 호  0553132500 40 0553132501
드0 한 인다
의 0 기 29 장 들 률 룰 1 빠 미 
후윅웅숍 중 악 
틱 교

Text Vector (First 10 values):
[1.]


In [56]:
import torch.nn.functional as F

class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2.0, weight=None):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha  # Focal Loss 
        self.gamma = gamma  # Focal Loss의 
        self.weight = weight  # Cross-Entropy 

    def forward(self, outputs, targets):
        # Cross-Entropy 
        cross_entropy_loss = F.cross_entropy(outputs, targets, weight=self.weight)

        # Focal Loss 
        ce_loss = F.cross_entropy(outputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)  # 예측 확률
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()

        total_loss = self.alpha * focal_loss + (1 - self.alpha) * cross_entropy_loss
        return total_loss


In [9]:
#Mixed Precision Training
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()  # Mixed Precision Training을 위한 스케일러 초기화

def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, text, targets in pbar:
        image, text, targets = image.to(device), text.to(device), targets.to(device)

        optimizer.zero_grad(set_to_none=True)
        with autocast():  # Mixed Precision 적용
            preds = model(image, text)
            loss = loss_fn(preds, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret


In [10]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '/root/data'

# model config
model_name = 'tf_efficientnetv2_m' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 288
LR = 1e-4
EPOCHS = 50
BATCH_SIZE = 32
num_workers = 2

In [11]:
# 훈련 데이터에 대한 Transform 코드
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.RandomResizedCrop(height=img_size, width=img_size, scale=(0.8, 1.2), ratio=(0.75, 1.33), p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=(-40,40), p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.3),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, min_holes=1, fill_value=0, p=0.5),
    A.MotionBlur(blur_limit=5, p=0.2),
    A.GaussianBlur(blur_limit=(3,7), p=0.2),
    A.GaussNoise(always_apply=False, var_limit=(50.0, 200.0), p=0.5, per_channel=True, mean= 0.0),
    A.Affine(shear=15, rotate=10, scale=(0.9, 1.1), p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
    A.GridDistortion(p=0.3),
    A.ImageCompression(quality_lower=70, quality_upper=100, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3), 
    A.GridDropout(ratio=0.5, holes_number_x=3, holes_number_y=3, p=0.3),  # GridMask
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),
    A.ChannelShuffle(p=0.1),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=0.2),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.2),  # 이미지 선명화 추가
    A.Normalize(mean=[0.5805, 0.5895, 0.5944], std=[0.186, 0.183, 0.187]),
    ToTensorV2(),
])

# 클래스 3과 7에 대한 transform (cutout 비율 0.4)
transform_p_0_4 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.RandomResizedCrop(height=img_size, width=img_size, scale=(0.8, 1.2), ratio=(0.75, 1.33), p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=(-40, 40), p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.3),
    # CoarseDropout의 p 값을 0.4로 설정
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, min_holes=1, fill_value=0, p=0.4),
    # 나머지 transform은 동일
    A.MotionBlur(blur_limit=5, p=0.2),
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),
    A.GaussNoise(var_limit=(50.0, 200.0), p=0.5, per_channel=True, mean=0.0),
    A.Affine(shear=15, rotate=10, scale=(0.9, 1.1), p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
    A.GridDistortion(p=0.3),
    A.ImageCompression(quality_lower=70, quality_upper=100, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3),
    A.GridDropout(ratio=0.5, holes_number_x=3, holes_number_y=3, p=0.3),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),
    A.ChannelShuffle(p=0.1),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=0.2),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.2),
    A.Normalize(mean=[0.5805, 0.5895, 0.5944], std=[0.186, 0.183, 0.187]),
    ToTensorV2(),
])

# 클래스 4와 14에 대한 transform (cutout 비율 0.45)
transform_p_0_45 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.RandomResizedCrop(height=img_size, width=img_size, scale=(0.8, 1.2), ratio=(0.75, 1.33), p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=(-40, 40), p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.3),
    # CoarseDropout의 p 값을 0.45로 설정
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, min_holes=1, fill_value=0, p=0.45),
    # 나머지 transform은 동일
    A.MotionBlur(blur_limit=5, p=0.2),
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),
    A.GaussNoise(var_limit=(50.0, 200.0), p=0.5, per_channel=True, mean=0.0),
    A.Affine(shear=15, rotate=10, scale=(0.9, 1.1), p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
    A.GridDistortion(p=0.3),
    A.ImageCompression(quality_lower=70, quality_upper=100, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3),
    A.GridDropout(ratio=0.5, holes_number_x=3, holes_number_y=3, p=0.3),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),
    A.ChannelShuffle(p=0.1),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=0.2),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.2),
    A.Normalize(mean=[0.5805, 0.5895, 0.5944], std=[0.186, 0.183, 0.187]),
    ToTensorV2(),
])

# 클래스별 transform을 딕셔너리로 정의
class_transforms = {
    3: transform_p_0_4,
    7: transform_p_0_4,
    4: transform_p_0_45,
    14: transform_p_0_45,
}

# 테스트 데이터에 대한 Transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.5805, 0.5895, 0.5944], std=[0.186, 0.183, 0.187]),
    ToTensorV2(),
])

In [12]:
# train.csv 파일 불러오기
train_df = pd.read_csv("/root/data/train.csv")

# 특정 행의 target 값 수정
train_df.loc[428, 'target'] = 7
train_df.loc[1095, 'target'] = 14
train_df.loc[862, 'target'] = 3
train_df.loc[192, 'target'] = 7
train_df.loc[1237, 'target'] = 14
train_df.loc[38, 'target'] = 10
train_df.loc[340, 'target'] = 10

# 변경된 DataFrame을 다시 train.csv에 저장
train_df.to_csv("/root/data/train.csv", index=False)

# Dataset 정의
trn_dataset = ImageDataset(
    "/root/data/train.csv",
    "/root/data/train/",
    transform=trn_transform,
    oversample=True
)
tst_dataset = ImageDataset(
    "/root/data/sample_submission.csv",
    "/root/data/test/",
    transform=tst_transform
)

# 데이터셋 크기 확인
print(len(trn_dataset), len(tst_dataset))

6936 3140


In [13]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=False,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    persistent_workers=False
)

In [14]:
class MultimodalModel(nn.Module):
    def __init__(self, model_name, text_dim=50, num_classes=17, text_weight=1.5):
        super(MultimodalModel, self).__init__()
        self.image_model = timm.create_model(model_name, pretrained=True, num_classes=0)  # 최종 분류 레이어 제거
        self.text_fc = nn.Linear(text_dim, 256)  # 텍스트 벡터를 위한 추가 레이어
        self.classifier = nn.Linear(256 + self.image_model.num_features, num_classes)  # 이미지와 텍스트 특징 결합
        self.text_weight = text_weight

    def forward(self, image, text):
        img_features = self.image_model(image)
        text_features = self.text_fc(text)
        
        # 이미지와 텍스트 특징을 결합하여 최종 분류 레이어에 전달
        combined_features = torch.cat([img_features, text_features], dim=1)
        return self.classifier(combined_features)


In [15]:
class_weights = torch.tensor([1.0] * 17).to(device) 
class_weights[[3, 4, 7, 14]] *= 2 

model = MultimodalModel(
    model_name=model_name,
    text_dim=50,  # 텍스트 벡터의 차원
    num_classes=17
).to(device)

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1,weight=class_weights)
#loss_fn = CombinedLoss(alpha=0.5, gamma=2.0, weight=class_weights).to(device)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR

# 조기 종료 설정
patience = 5  # 개선되지 않는 에포크 수
best_f1 = 0   # 최고 F1 스코어
early_stopping_counter = 0  # 조기 종료 카운터

# 학습률 스케줄러 설정
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)
#scheduler = OneCycleLR(optimizer, max_lr=LR, steps_per_epoch=len(trn_loader), epochs=EPOCHS)

# 학습 루프
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    text_weight = 1.5  # 특정 클래스에서 텍스트 비중을 높이는 값

    for images, text_vectors, targets in trn_loader:
        images, text_vectors, targets = images.to(device), text_vectors.to(device), targets.to(device)
        
        optimizer.zero_grad()

        # 특정 클래스의 경우 텍스트 비중을 높임
        mask = torch.isin(targets, torch.tensor([3, 4, 7, 14], device=device))
        text_vectors = torch.where(mask.unsqueeze(1), text_vectors * text_weight, text_vectors)

        # 모델에 이미지와 텍스트 벡터를 전달
        outputs = model(images, text_vectors)
        loss = loss_fn(outputs, targets)
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(outputs.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

    train_loss /= len(trn_loader)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    # Scheduler step을 F1 스코어 기준으로 조정
    scheduler.step(train_f1)

    # F1 스코어 개선 확인
    if train_f1 > best_f1:
        best_f1 = train_f1  # 최고 F1 갱신
        early_stopping_counter = 0  # 카운터 초기화
    else:
        early_stopping_counter += 1  # 개선되지 않으면 카운터 증가

    # 조기 종료 조건 확인
    if early_stopping_counter >= patience:
        print("Early stopping due to no improvement in F1 score.")
        break  # 학습 중단

    # 로그 출력
    log = f"Epoch {epoch + 1}\nTrain Loss: {train_loss:.4f}\nTrain F1 Score: {train_f1:.4f}\n"
    print(log)


Epoch 1
Train Loss: 1.5258
Train F1 Score: 0.6537

Epoch 2
Train Loss: 0.9168
Train F1 Score: 0.8940

Epoch 3
Train Loss: 0.8237
Train F1 Score: 0.9258

Epoch 4
Train Loss: 0.7697
Train F1 Score: 0.9431

Epoch 5
Train Loss: 0.7277
Train F1 Score: 0.9565

Epoch 6
Train Loss: 0.7041
Train F1 Score: 0.9644

Epoch 7
Train Loss: 0.6859
Train F1 Score: 0.9705

Epoch 8
Train Loss: 0.6694
Train F1 Score: 0.9735

Epoch 9
Train Loss: 0.6536
Train F1 Score: 0.9792

Epoch 10
Train Loss: 0.6573
Train F1 Score: 0.9771

Epoch 11
Train Loss: 0.6475
Train F1 Score: 0.9798

Epoch 12
Train Loss: 0.6302
Train F1 Score: 0.9859

Epoch 13
Train Loss: 0.6367
Train F1 Score: 0.9845

Epoch 14
Train Loss: 0.6277
Train F1 Score: 0.9850

Epoch 15
Train Loss: 0.6183
Train F1 Score: 0.9886

Epoch 16
Train Loss: 0.6311
Train F1 Score: 0.9847

Epoch 17
Train Loss: 0.6277
Train F1 Score: 0.9866

Epoch 00018: reducing learning rate of group 0 to 5.0000e-05.
Epoch 18
Train Loss: 0.6185
Train F1 Score: 0.9876

Epoch 19
Tr

In [19]:
preds_list = []

model.eval()
for image, _, _ in tqdm(tst_loader):   # tst_loader에서는 image와 target만 반환
    image = image.to(device)

    # 더미 텍스트 벡터 생성 (예: 크기 50의 제로 텐서)
    text = torch.zeros((image.size(0), 50)).to(device)

    with torch.no_grad():
        preds = model(image, text)  # 더미 텍스트 벡터도 함께 전달
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:22<00:00,  4.31it/s]


In [20]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [21]:
sample_submission_df = pd.read_csv("/root/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [22]:
pred_df.to_csv("/root/data/output.csv", index=False)

In [23]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
